In [1]:
# Import Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
import statsmodels.api as sm 
import statsmodels.tools
import seaborn as sns #pretty visualisations
import matplotlib.pyplot as plt
import re

In [2]:
# Read in data
df = pd.read_csv('Life Expectancy Data.csv')

In [ ]:
df.head()

In [ ]:
# Check for nulls
df.isnull().sum()

In [ ]:
# Check Datatypes 
df.dtypes

In [ ]:
# Check number of observations
df.shape

In [ ]:
df.describe()

## Train Test Split

In [3]:
# Allocate features and target column
feature_cols = list(df.columns)
feature_cols.remove('Life_expectancy')

In [4]:
# Assign to X and y
X = df[feature_cols]
y = df['Life_expectancy']

In [5]:
# Train test split the data
X_train, X_test, y_train, y_test = train_test_split (X,  y, test_size = 0.2, random_state = 10)

In [6]:
# check split worked
print(X_train.shape)
print(y_test.shape)

(2291, 20)
(573,)


In [7]:
#X_train.shape[0] == y_train.shape[0]
X_test.shape[0] == y_test.shape[0]

True

In [ ]:
X_train.head()

## EDA

In [ ]:
df.describe()

In [ ]:
## Correlations
sns.heatmap(df.corr(numeric_only = True), 
            annot = True,
           cmap = 'rocket_r',
           vmin = -1,
           vmax = 1,
           fmt = '.2',
           mask = np.triu(df.corr(numeric_only = True)))

plt.show()

### BMI

In [ ]:
# Scatterplot of BMI vs Life Expectancy
sns.scatterplot(data = df, x = 'Life_expectancy', y= 'BMI')
plt.show()

# Boxplot of BMI 
plt.figure(figsize = (20, 5))
sns.boxplot(data = df['BMI'],
            orient = 'h')
plt.show()

### GDP_per_capita
* Exponential curve
* Will need transforming - log

In [ ]:
sns.scatterplot(data = df, x= 'Life_expectancy', y = 'GDP_per_capita')
plt.show()

### Schooling
* Relatively strong correlation

In [ ]:
sns.scatterplot(data =df, x ='Life_expectancy',y='Schooling')
plt.show()

### Polio

* unsure on whether this is linear really?
* Some outliers

In [ ]:
sns.scatterplot(data = df, x = 'Life_expectancy', y= 'Polio')
plt.show()

plt.figure(figsize = (20, 5))
sns.boxplot(data = df['Polio'],
            orient = 'h')
plt.show()

### Strong correlation between Diphtheria and Polio 

In [ ]:
sns.scatterplot(data = df, x= 'Diphtheria', y ='Polio')
plt.show()

### Strong correlation between thinnness of 5-9yrs and 10-19yrs

* Linear relationship
* Some outliers in both variables
* Potential of Multicollinearity

In [ ]:
sns.scatterplot(data = df, x ='Thinness_ten_nineteen_years', y= 'Thinness_five_nine_years')
plt.show()

#boxplot thinness 10-19
plt.figure(figsize = (20, 5))
sns.boxplot(data = df['Thinness_ten_nineteen_years'],
            orient = 'h')
plt.show()

# boxplot for thinness 5-9
plt.figure(figsize = (20, 5))
sns.boxplot(data = df['Thinness_five_nine_years'],
            orient = 'h')
plt.show()

## Feature Engineering 

In [ ]:
def feature_eng (df):
    df = df.copy()

    #removing columns
    df = df.drop(columns = ['Country'])
    df = df.drop(columns = ['Economy_status_Developing'])
    df = df.drop(columns = ['Infant_deaths'])    
    
    #calculated field
    df['avg_thin'] = (df['Thinness_five_nine_years'] + df['Thinness_ten_nineteen_years']) / 2
    
    #one hot encoding method
    df = pd.get_dummies(df, columns = ['Region'], drop_first = True, prefix = 'Region', dtype=int)
    
    #rename region columns
    df = df.rename(columns = {'Region_Asia' : 'Region_1',
                              'Region_Central America and Caribbean' : 'Region_2',
                              'Region_European Union' : 'Region_3',
                              'Region_Middle East' : 'Region_4' ,
                              'Region_North America' : 'Region_5',
                              'Region_Oceania' : 'Region_6',
                              'Region_Rest of Europe' : 'Region_7',
                              'Region_South America': 'Region_8'})
    
    # logging exponential data
    df['GDP_log'] = np.log(df['GDP_per_capita'])
    
    #scaling
    scaler = StandardScaler()
    df[df.columns] = scaler.fit_transform(df[df.columns])
    
    df = sm.add_constant(df)
    return df

In [ ]:
X_train_fe = feature_eng(X_train)

In [ ]:
X_train_fe.head()

In [ ]:
# Check no nulls in our engineered data
X_train_fe.isnull().sum()

In [ ]:
# Check datatypes of our engineered data
X_train_fe.dtypes

In [ ]:
plt.figure(figsize = (11, 11))
sns.heatmap(X_train_fe.corr(numeric_only = True), 
            annot = True,
            cmap = 'rocket_r',
            vmin = -1,
            vmax = 1,
            fmt = '.1',
            mask = np.triu(X_train_fe.corr(numeric_only = True)))
 
plt.show()

In [ ]:
#remove population, measles - high p-values
#remove alcohol consumptoin reduced region_middle_east's p-value!!

feature_cols = ['const', 'Year','Under_five_deaths', 'Adult_mortality','GDP_log',
                #'Measles', 'population_mln', 'Alcohol_consumption',
                  'Polio','Diphtheria','BMI','Hepatitis_B',
                 'Incidents_HIV', 'avg_thin', 'GDP_per_capita',
                 'Schooling', 'Economy_status_Developed', 'Region_1',
                 'Region_2','Region_3', 'Region_4','Region_5','Region_6','Region_7','Region_8']


In [ ]:
lin_reg = sm .OLS (y_train, X_train_fe [feature_cols])
results = lin_reg.fit()
results.summary()

In [ ]:
y_pred = results.predict(X_train_fe[feature_cols])
rmse = statsmodels.tools.eval_measures.rmse(y_train,y_pred)
print(rmse)


In [ ]:
X_test_fe = feature_eng(X_test)
X_test_fe = X_test_fe[feature_cols]

In [ ]:
y_test_pred = results.predict(X_test_fe)
rmse = statsmodels.tools.eval_measures.rmse(y_test, y_test_pred)
print(rmse)

### BINS

In [ ]:
df['Adult_mortality'].describe()

In [ ]:
df

## Sensitive Model

In [17]:
def feature_eng (df):
    df = df.copy()

    #removing columns
    df = df.drop(columns = ['Country'])
    df = df.drop(columns = ['Economy_status_Developing'])
    df = df.drop(columns = ['Infant_deaths'])   
    
    #calculated field
    df['avg_thin'] = (df['Thinness_five_nine_years'] + df['Thinness_ten_nineteen_years']) / 2
    
    ## Bins for adult mortality
    # group adult mortalities into low, medium, and high
    low_percentile = df["Adult_mortality"].quantile(0.33)
    high_percentile = df["Adult_mortality"].quantile(0.66)

    # Bin function based on percentiles for adult mortalities
    def adult_mortality_bins(value):
        if value <= low_percentile:
            return 'Low'
        elif value <= high_percentile:
            return 'Medium'
        else:
            return 'High'
    
    #apply bin function to adult mortalities
    df["Adult_mortality_bins"] = df["Adult_mortality"].apply(adult_mortality_bins)
    
    ## Bins for Under_five_deaths
    # group adult mortalities into low, medium, and high
    low_percentile = df["Under_five_deaths"].quantile(0.33)
    high_percentile = df["Under_five_deaths"].quantile(0.66)

    # Bin function based on percentiles for infant mortalities
    def under5_deaths_bins(value):
        if value <= low_percentile:
            return 'Low'
        elif value <= high_percentile:
            return 'Medium'
        else:
            return 'High'
    
    #apply bin function to under 5 deaths
    df['Under_five_deaths_bins'] = df['Under_five_deaths'].apply(under5_deaths_bins)
    
    #one hot encoding method
    df = pd.get_dummies(df, columns = ['Region'], drop_first = True, prefix = 'Region', dtype=int)
    df = pd.get_dummies(df, columns = ['Adult_mortality_bins'], drop_first = True, prefix = 'Adult_mortality', dtype=int)
    df = pd.get_dummies(df, columns = ['Under_five_deaths_bins'], drop_first = True, prefix = 'Under_five_deaths', dtype=int)
    
    #rename region columns
    df = df.rename(columns = {'Region_Asia' : 'Region_1',
                              'Region_Central America and Caribbean' : 'Region_2',
                              'Region_European Union' : 'Region_3',
                              'Region_Middle East' : 'Region_4' ,
                              'Region_North America' : 'Region_5',
                              'Region_Oceania' : 'Region_6',
                              'Region_Rest of Europe' : 'Region_7',
                              'Region_South America': 'Region_8'})
    
    # logging exponential data
    df['GDP_log'] = np.log(df['GDP_per_capita'])
    
    #scaling
    scaler = StandardScaler()
    df[df.columns] = scaler.fit_transform(df[df.columns])
    
    df = sm.add_constant(df)
    return df

In [19]:
X_train_fe = feature_eng(X_train)

In [21]:
X_train_fe.head()

,const,Year,Under_five_deaths,Adult_mortality,Alcohol_consumption,Hepatitis_B,Measles,BMI,Polio,Diphtheria,...,Region_4,Region_5,Region_6,Region_7,Region_8,Adult_mortality_Low,Adult_mortality_Medium,Under_five_deaths_Low,Under_five_deaths_Medium,GDP_log
1680,1.0,-0.085510,0.137039,-0.351286,-1.083147,-0.020056,0.677760,-0.114456,-0.097782,-0.020917,...,-0.286691,-0.126351,-0.258979,-0.309270,-0.272166,-0.701789,1.424929,-0.702482,-0.701789,-1.289892
2675,1.0,-0.949477,2.151642,1.393963,0.383955,0.169292,-1.085028,-1.963016,-0.758410,-0.279951,...,-0.286691,-0.126351,-0.258979,-0.309270,-0.272166,-0.701789,-0.701789,-0.702482,-0.701789,-1.884274
2247,1.0,1.426430,-0.767615,-0.460535,1.372928,0.674219,0.624343,0.578754,0.100406,0.108600,...,-0.286691,-0.126351,-0.258979,-0.309270,-0.272166,-0.701789,1.424929,1.423524,-0.701789,0.295892
2088,1.0,0.778456,-0.893951,-1.106856,0.401571,0.232408,0.891432,0.624968,0.496783,0.497150,...,-0.286691,-0.126351,-0.258979,3.233419,-0.272166,1.424929,-0.701789,1.423524,-0.701789,1.943239
2331,1.0,1.426430,1.219916,1.172945,0.955194,-1.976650,-0.711103,-0.530382,-2.079666,-4.294968,...,-0.286691,-0.126351,-0.258979,-0.309270,-0.272166,-0.701789,-0.701789,-0.702482,-0.701789,0.744062


In [ ]:
# Check no nulls in our engineered data
X_train_fe.isnull().sum()

In [ ]:
X_train_fe.dtypes

In [79]:
#minimal model              
feature_cols = ['const', 'Year',
                 'Population_mln',
                 'GDP_per_capita','GDP_log', 
                 'Economy_status_Developed',
                 'Adult_mortality_Low', 'Adult_mortality_Medium',
                 'Under_five_deaths_Low','Under_five_deaths_Medium',
                 #'Under_five_deaths', 'Adult_mortality', -sensitive data
                 #'Measles', 
                 'Incidents_HIV',
                 #'BMI', - high p_value
                 #'Schooling', #-high p_value
                 #'avg_thin',
                 'Alcohol_consumption',
                 'Polio','Hepatitis_B','Diphtheria',
                 'Region_1', 'Region_2','Region_3', 'Region_4','Region_5','Region_6','Region_7','Region_8']


In [81]:
lin_reg = sm .OLS (y_train, X_train_fe [feature_cols])
results = lin_reg.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        Life_expectancy   R-squared:                       0.934
Model:                            OLS   Adj. R-squared:                  0.933
Method:                 Least Squares   F-statistic:                     1523.
Date:                Thu, 05 Dec 2024   Prob (F-statistic):               0.00
Time:                        13:43:44   Log-Likelihood:                -5272.6
No. Observations:                2291   AIC:                         1.059e+04
Df Residuals:                    2269   BIC:                         1.072e+04
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                       68.8680      0.051   1357.264      0.000      68.769      68.968
Year                         0.6341      0.053     12.074      0.000       0.531       0.737
Population_mln              -0.0307      0.055     -0.558      0.577      -0.138       0.077
GDP_per_capita              -0.1907      0.103     -1.858      0.063      -0.392       0.011
GDP_log                      2.6412      0.147     17.970      0.000       2.353       2.929
Adult_mortality_Low          3.1876      0.115     27.652      0.000       2.962       3.414
Adult_mortality_Medium       1.8124      0.091     19.899      0.000       1.634       1.991
Under_five_deaths_Low        1.3771      0.147      9.375      0.000       1.089       1.665
Under_five_deaths_Medium     1.1314      0.110     10.277      0.000       0.916       1.347
Incidents_HIV               -2.1639      0.063    -34.362      0.000      -2.287      -2.040
Alcohol_consumption         -0.2884      0.091     -3.174      0.002      -0.467      -0.110
Polio                        1.1110      0.178      6.236      0.000       0.762       1.460
Hepatitis_B                 -0.3777      0.082     -4.585      0.000      -0.539      -0.216
Diphtheria                   0.7201      0.186      3.878      0.000       0.356       1.084
Region_1                     1.0503      0.070     14.954      0.000       0.913       1.188
Region_2                     1.0029      0.073     13.738      0.000       0.860       1.146
Region_3                     1.2284      0.104     11.828      0.000       1.025       1.432
Region_4                     0.3919      0.070      5.579      0.000       0.254       0.530
Region_5                     0.2915      0.058      4.985      0.000       0.177       0.406
Region_6                     0.5671      0.061      9.265      0.000       0.447       0.687
Region_7                     1.0074      0.076     13.173      0.000       0.857       1.157
Region_8                     0.9164      0.067     13.729      0.000       0.785       1.047
==============================================================================
Omnibus:                      125.247   Durbin-Watson:                   1.955
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              250.297
Skew:                          -0.379   Prob(JB):                     4.45e-55
Kurtosis:                       4.431   Cond. No.                         11.1
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [77]:
y_pred = results.predict(X_train_fe[feature_cols])
rmse = statsmodels.tools.eval_measures.rmse(y_train,y_pred)
print(rmse)

2.3604515324830966


In [67]:
X_test_fe = feature_eng(X_test)
X_test_fe = X_test_fe[feature_cols]

In [59]:
y_test_pred = results.predict(X_test_fe)
rmse = statsmodels.tools.eval_measures.rmse(y_test, y_test_pred)
print(rmse)

3.058012742115338
